In [86]:
!pip install pillow-heif
import torch
import requests
from transformers import SamModel, SamProcessor, CLIPProcessor, CLIPModel
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
import os
import pandas as pd

#for authing drive, and file hadnling on drive
import gspread
from google.auth import default
from google.colab import auth

from googleapiclient.http import MediaIoBaseDownload
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

#for opening images, and heic files
from PIL import Image
from pillow_heif import register_heif_opener


from sklearn.model_selection import train_test_split
drive.mount('/content/drive')
device = "cuda" if torch.cuda.is_available() else "cpu"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 63.5 MB/s eta 0:00:00
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


google colab accessing google sheets example notebook: https://colab.research.google.com/notebooks/io.ipynb#scrollTo=vz-jH8T_Uk2c

In [79]:
#pipeline code
#TODO: test it in local to see if it works with pathing and auth
class Gsheet_data_download_split:
  #paths should be paths from google colab to google drive
  def __init__(self, gSheet_path, download_dir_path):
    self.gSheet_path = gSheet_path
    self.download_dir_path = download_dir_path
    drive.mount('/content/drive')

    #auth_obj for reading into google sheet, pydrive obj for downladoing with img links
    self.auth_obj, self.pydrive_obj = self.auth_user()
    self.sheet_obj = self.connect_to_gsheet()

  #don't use this publicly haha
  def auth_user(self):
    try:
      auth.authenticate_user()
      #use this to get current user's credentials, for google colab
      creds, _ = default()
      #create auth obj for g sheet python api for google colab
      gc = gspread.authorize(creds)

      #need to also inst pydrive wrapper obj for accessing img links:
      gauth = GoogleAuth()
      gauth.credentials = GoogleCredentials.get_application_default()
      drive_obj = GoogleDrive(gauth)

      return gc, drive_obj
    except Exception as e:
      print(f"Error from auth_user of Gsheet_data_download_split: {e}")
      return False

  def connect_to_gsheet(self):
    try:
      connected_sheet = self.auth_obj.open_by_url(self.gSheet_path)
      self.connected_to_gsheet = True
      return connected_sheet
    except Exception as e:
      print(f"Error from connect_to_gsheet of Gsheet_data_download_split: {e}")
      return False

  ###+++++
  # user callable functions
  ###+++++
  def read_sheet_into_pd(self):
    try:
      worksheet = self.sheet_obj.get_worksheet(0)

      # get_all_values gives a list of rows.
      rows = worksheet.get_all_values()
      print(rows)
      sheet_df = pd.DataFrame.from_records(rows)
      return sheet_df
    except Exception as e:
      print(f"Error from read_sheet_into_pd of Gsheet_data_download_split: {e}")
      return False

  def get_imgs_and_split(self):
    try:
      sheet_df = self.read_sheet_into_pd()
      train_df, test_df = train_test_split(sheet_df, test_size=0.2)
      return train_df, test_df
    except Exception as e:
      print(f"Error from get_imgs_and_split of Gsheet_data_download_split: {e}")
      return False

  #helper func for downloading one df worth of imgs
  def download_imgs_from_one_df(self, input_df, download_path):
    try:
        count = 0
        for index, row in input_df.iterrows():
            file_id = row[1]
            file_id = file_id.split('id=')[-1] #must do this since id is embedded in the url
            stave_count = row[2]
            try:
              #need to use pydrive object to create the file object first, which is then used for downloading to local
              file_obj = self.pydrive_obj.CreateFile({'id': file_id})

              file_metaData = file_obj.FetchMetadata(fetch_all=True)
              file_name = file_obj['title']

              #GetContentFile() for downloading from drive to local
              file_obj.GetContentFile(os.path.join(download_path, f"{file_name}.jpg"))

              with open(os.path.join(download_path, f"staves_count.txt"), 'a') as f:
                f.write(f"{file_name}:{str(stave_count)}")
                f.write("\n")

              print(f"Successfully downloaded {file_id}.jpg to {download_path}")
              count += 1
              if count > 30:
                return
            except Exception as e:
              print(f"Failed to download file {file_id}: {e}")

    except Exception as e:
        print(f"An error occurred while downloading the Google Drive image: {e}")

  def download_imgs_from_split_dfs(self):
    try:
      train_df, test_df = self.get_imgs_and_split()
      self.download_imgs_from_one_df(train_df, os.path.join(self.download_dir_path, "train"))
      self.download_imgs_from_one_df(test_df, os.path.join(self.download_dir_path, "test"))
    except Exception as e:
      print(f"Error from download_imgs_from_split_dfs of Gsheet_data_download_split: {e}")
      return False

In [80]:
SHEET_PATH=""
DOWNLOAD_PATH="/content/drive/MyDrive/!mlData/stavesProjectData"

In [81]:
!rm -rf /content/drive/MyDrive/!mlData/stavesProjectData/train/* /content/drive/MyDrive/!mlData/stavesProjectData/test/*

In [104]:
gsheetObj = Gsheet_data_download_split(gSheet_path=SHEET_PATH, download_dir_path=DOWNLOAD_PATH)
gsheetObj.download_imgs_from_split_dfs()

In [87]:
img_list = []

#need this since some files does not seem to be saved correctly as jpg but as heic
register_heif_opener()

for anImg in os.listdir('/content/drive/MyDrive/!mlData/stavesProjectData/test'):
  if anImg.endswith('.jpg'):
    img_list.append(Image.open(os.path.join('/content/drive/MyDrive/!mlData/stavesProjectData/test', anImg)))

staves_count_list = []
with open('/content/drive/MyDrive/!mlData/stavesProjectData/test/staves_count.txt', 'r') as f:
  for line in f:
    staves_count_list.append(int(line.split(':')[-1].strip()))

print(staves_count_list)

[649, 627, 675, 667, 705, 659, 625, 652, 679, 654, 629, 773, 643, 620, 788, 674, 773, 655, 686, 690, 653]


In [88]:
#laod from huggingface
model_clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor_clip = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

pytorch_model.bin:  45%|####5     | 273M/605M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [89]:
#note: change the name of these 2 variable below in both lines 10 and 2 in the cell below to get the correct predicted class
clip_numerical_classes = [str(i)+" staves" for i in range(100, 1000)]
clip_object_classes = ["staves", "car", "human", "factory"]

In [94]:
#get text-img embedding stuff
predicted_categories=[]
for anImg, cur_stave_count in zip(img_list, staves_count_list):
  inputs = processor_clip(text=clip_numerical_classes, images=anImg, return_tensors="pt", padding=True)
  inputs = {key: value.to(device) for key, value in inputs.items()}
  with torch.no_grad():
      outputs = model_clip(**inputs)
  #get logies
  logits_per_image = outputs.logits_per_image
  #need this for extracting top classes
  predicted_indices = torch.argmax(logits_per_image, dim=1)
  cur_prediction=[clip_numerical_classes[idx] for idx in predicted_indices]
  predicted_categories.append(cur_prediction[0])

In [101]:
clip_results_df = pd.DataFrame({
    "predicted_category": predicted_categories,
    "correct_stave_count": staves_count_list,
})

clip_results_df['predicted_category_asInt'] = clip_results_df['predicted_category'].str.replace(' staves', '').astype(int)
clip_results_df['accuracy'] = (abs(clip_results_df['predicted_category_asInt'] - clip_results_df['correct_stave_count'])) / clip_results_df['correct_stave_count']
clip_results_df['accuracy'] = 1 - clip_results_df['accuracy']
clip_results_df

,predicted_category,correct_stave_count,predicted_category_asInt,accuracy
0,580 staves,649,580,0.893683
1,650 staves,627,650,0.963317
2,450 staves,675,450,0.666667
3,205 staves,667,205,0.307346
4,195 staves,705,195,0.276596
5,650 staves,659,650,0.986343
6,580 staves,625,580,0.928000
7,480 staves,652,480,0.736196
8,205 staves,679,205,0.301915
9,195 staves,654,195,0.298165


In [103]:
print(f"avg accuracy: {clip_results_df['accuracy'].mean()}")

avg accuracy: 0.5846252737364053
